In [ ]:
import os
import meshio
import polyscope as ps
import polyscope.imgui as psim
import numpy as np
from collections import Counter
import trimesh
from trimesh.collision import CollisionManager

In [ ]:
# Paths to the mesh files
MESH_FILE1 = os.path.join(os.getcwd(), "meshes/Cube.msh")
MESH_FILE2 = os.path.join(os.getcwd(), "meshes/Cube.msh")

In [ ]:
# Color Definitions
BOUNDARY_COLOR = [1.0, 0.0, 0.0]
SURFACE_COLOR1 = [0.0, 0.0, 1.0]  # Blue for Mesh 1
SURFACE_COLOR2 = [1.0, 0.0, 0.0]  # Red for Mesh 2
INTERSECT_COLOR = [1.0, 1.0, 0.0]  # Yellow for Intersection
INTERSECT_COLOR1 = [0.0, 1.0, 1.0]  # Cyan for Mesh 1 intersect
INTERSECT_COLOR2 = [1.0, 1.0, 0.0]  # Yellow for Mesh 2 intersect

# Transformation Parameters
scale1 = [1.0, 1.0, 1.0]
rotation1 = [0.0, 0.0, 0.0]
translation1 = [0.0, 0.0, 0.0]

scale2 = [1.0, 1.0, 1.0]
rotation2 = [0.0, 0.0, 0.0]
translation2 = [0.0, 0.0, 0.0]

# Global variables for Polyscope
points1 = None
points2 = None
tetra_cells1 = None
tetra_cells2 = None
surface_faces1 = None
surface_faces2 = None
show_intersection_box = False

In [ ]:
def verify_mesh_files_exist(*file_paths):
    """Verify that all provided mesh files exist."""
    for file_path in file_paths:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Mesh file not found: {file_path}")

In [ ]:
def load_mesh(file_path):
    """Load a mesh file using meshio and return its points and tetrahedral cells."""
    mesh = meshio.read(file_path)
    points = mesh.points
    tetra_cells = next(
        (cell_block.data for cell_block in mesh.cells if cell_block.type.lower() == "tetra"),
        None
    )
    if tetra_cells is None:
        raise ValueError(f"No tetrahedral cells found in mesh: {file_path}")
    return points, tetra_cells

In [ ]:
def apply_transform(points, scale, rotation, translation):
    """Apply scaling, rotation, and translation to the points."""
    # Scaling
    p = points * scale

    # Rotation
    rx, ry, rz = np.radians(rotation)
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(rx), -np.sin(rx)],
                   [0, np.sin(rx), np.cos(rx)]])
    Ry = np.array([[np.cos(ry), 0, np.sin(ry)],
                   [0, 1, 0],
                   [-np.sin(ry), 0, np.cos(ry)]])
    Rz = np.array([[np.cos(rz), -np.sin(rz), 0],
                   [np.sin(rz), np.cos(rz), 0],
                   [0, 0, 1]])
    R = Rz @ Ry @ Rx
    p = p @ R.T

    # Translation
    p += translation
    return p

In [ ]:
def extract_boundary_faces(tetra_cells):
    """Extract boundary faces from tetrahedral cells."""
    all_faces = []
    for c in tetra_cells:
        faces = [
            tuple(sorted([c[0], c[1], c[2]])),
            tuple(sorted([c[0], c[1], c[3]])),
            tuple(sorted([c[0], c[2], c[3]])),
            tuple(sorted([c[1], c[2], c[3]])),
        ]
        all_faces.extend(faces)
    face_count = Counter(all_faces)
    boundary_faces = [face for face, count in face_count.items() if count == 1]
    return np.array(boundary_faces)

In [ ]:
def extract_internal_tetrahedra(tetra_cells, boundary_faces):
    """Extract internal tetrahedra by removing tetrahedra with boundary faces."""
    boundary_set = set(tuple(sorted(face)) for face in boundary_faces)
    internal_tetrahedra = []
    for tet in tetra_cells:
        faces = [
            tuple(sorted([tet[0], tet[1], tet[2]])),
            tuple(sorted([tet[0], tet[1], tet[3]])),
            tuple(sorted([tet[0], tet[2], tet[3]])),
            tuple(sorted([tet[1], tet[2], tet[3]])),
        ]
        if not any(face in boundary_set for face in faces):
            internal_tetrahedra.append(tet)
    return np.array(internal_tetrahedra)


In [ ]:
def compute_triangles_aabb(mesh):
    """
    Compute axis-aligned bounding boxes for each triangle in the mesh.
    
    Parameters:
        mesh (trimesh.Trimesh): The input mesh.
        
    Returns:
        np.ndarray: An array of shape (n_faces, 2, 3) where each entry contains the min and max
                    coordinates of the corresponding triangle.
    """
    min_coords = np.min(mesh.triangles, axis=1)
    max_coords = np.max(mesh.triangles, axis=1)
    return np.stack((min_coords, max_coords), axis=1)


In [ ]:
def find_surface_intersecting_triangles(mesh1, mesh2, boundary_faces1, boundary_faces2):
    """
    Find intersecting triangle pairs between the surfaces of two meshes.
    Only surface triangles (boundary faces) are considered.
    """
    surface_mesh1 = trimesh.Trimesh(vertices=mesh1.vertices, faces=boundary_faces1, process=False)
    surface_mesh2 = trimesh.Trimesh(vertices=mesh2.vertices, faces=boundary_faces2, process=False)

    manager1 = CollisionManager()
    manager1.add_object("mesh1", surface_mesh1)
    manager2 = CollisionManager()
    manager2.add_object("mesh2", surface_mesh2)

    if not manager1.in_collision_other(manager2):
        print("Surface meshes do not collide.")
        return []

    print("Surface meshes collide. Proceeding to find intersecting triangles...")

    mesh1_aabbs = compute_triangles_aabb(surface_mesh1)
    mesh2_aabbs = compute_triangles_aabb(surface_mesh2)

    intersecting_triangles = []

    for idx1, aabb1 in enumerate(mesh1_aabbs):
        overlap = (
            np.all(aabb1[0] <= mesh2_aabbs[:, 1], axis=1)
            & np.all(aabb1[1] >= mesh2_aabbs[:, 0], axis=1)
        )
        candidate_idxs2 = np.where(overlap)[0]

        if len(candidate_idxs2) == 0:
            continue

        tri1 = surface_mesh1.triangles[idx1]
        tris2 = surface_mesh2.triangles[candidate_idxs2]

        intersects = trimesh.geometry.triangles_intersect(tris2, tri1[np.newaxis, :])

        for i, does_intersect in enumerate(intersects):
            if does_intersect:
                idx2 = candidate_idxs2[i]
                intersecting_triangles.append((idx1, idx2))

    return intersecting_triangles

In [ ]:
def register_intersection_highlights(mesh1, mesh2, intersecting_pairs, color1=INTERSECT_COLOR1, color2=INTERSECT_COLOR2):
    """Register intersecting triangles from both meshes as separate surface meshes in Polyscope."""
    if ps.has_surface_mesh("Intersecting Triangles Mesh 1"):
        ps.remove_surface_mesh("Intersecting Triangles Mesh 1")
    if ps.has_surface_mesh("Intersecting Triangles Mesh 2"):
        ps.remove_surface_mesh("Intersecting Triangles Mesh 2")
    
    # Extract intersecting triangles from Mesh 1
    if intersecting_pairs:
        faces1 = [pair[0] for pair in intersecting_pairs]
        mesh1_intersect = trimesh.Trimesh(vertices=mesh1.vertices, faces=mesh1.faces[faces1], process=False)
        ps.register_surface_mesh("Intersecting Triangles Mesh 1", mesh1_intersect.vertices, mesh1_intersect.faces, color=color1)
        
        # Extract intersecting triangles from Mesh 2
        faces2 = [pair[1] for pair in intersecting_pairs]
        mesh2_intersect = trimesh.Trimesh(vertices=mesh2.vertices, faces=mesh2.faces[faces2], process=False)
        ps.register_surface_mesh("Intersecting Triangles Mesh 2", mesh2_intersect.vertices, mesh2_intersect.faces, color=color2)
    else:
        # No intersecting triangles to register
        pass

In [ ]:
def compute_bounding_box(vertices):
    """Compute the axis-aligned bounding box from a set of vertices."""
    min_corner = np.min(vertices, axis=0)
    max_corner = np.max(vertices, axis=0)
    return min_corner, max_corner

In [ ]:
def register_intersections(intersect_points, name="Intersection Points", color=INTERSECT_COLOR):
    """Register intersection points in Polyscope."""
    if ps.has_point_cloud(name):
        ps.remove_point_cloud(name)
    ps.register_point_cloud(name, intersect_points, color=color)

In [ ]:
def register_bounding_box(min_corner, max_corner, name, color):
    """Register a bounding box as a curve network in Polyscope."""
    # Ensure min_corner and max_corner are NumPy arrays
    min_corner = np.asarray(min_corner)
    max_corner = np.asarray(max_corner)

    if min_corner.shape != (3,) or max_corner.shape != (3,):
        raise ValueError("min_corner and max_corner must be 1D arrays of length 3.")

    # Define the 8 corners of the bounding box
    corners = np.array([
        [min_corner[0], min_corner[1], min_corner[2]],
        [min_corner[0], min_corner[1], max_corner[2]],
        [min_corner[0], max_corner[1], min_corner[2]],
        [min_corner[0], max_corner[1], max_corner[2]],
        [max_corner[0], min_corner[1], min_corner[2]],
        [max_corner[0], min_corner[1], max_corner[2]],
        [max_corner[0], max_corner[1], min_corner[2]],
        [max_corner[0], max_corner[1], max_corner[2]],
    ])

    # Define the edges connecting the corners
    edges = np.array([
        [0, 1], [0, 2], [0, 4],
        [1, 3], [1, 5],
        [2, 3], [2, 6],
        [3, 7],
        [4, 5], [4, 6],
        [5, 7],
        [6, 7],
    ])

    if ps.has_curve_network(name):
        ps.remove_curve_network(name)
    ps.register_curve_network(name, corners, edges, color=color)


In [ ]:
def remove_bounding_box(name):
    """Remove a bounding box if it exists."""
    if ps.has_curve_network(name):
        ps.remove_curve_network(name)

In [ ]:
def register_volume_mesh(name, points, tetra_cells, color):
    """Register a volume mesh in Polyscope."""
    if ps.has_volume_mesh(name):
        ps.remove_volume_mesh(name)
    ps.register_volume_mesh(name, points, tetra_cells, color=color)

In [ ]:
def update_geometry():
    """Update geometry based on transformations and compute intersections."""
    global points1_transformed, points2_transformed

    # Apply transformations
    points1_transformed = apply_transform(points1, scale1, rotation1, translation1)
    points2_transformed = apply_transform(points2, scale2, rotation2, translation2)

    # Update Polyscope surface meshes
    ps.get_surface_mesh("External Surface Mesh 1").update_vertex_positions(points1_transformed)
    ps.get_surface_mesh("External Surface Mesh 2").update_vertex_positions(points2_transformed)

    if show_intersection_box:
        # Create trimesh objects for the transformed meshes
        mesh1 = trimesh.Trimesh(vertices=points1_transformed, faces=surface_faces1, process=False)
        mesh2 = trimesh.Trimesh(vertices=points2_transformed, faces=surface_faces2, process=False)

        # Find intersecting triangles
        intersecting_pairs = find_intersecting_triangles_optimized(mesh1, mesh2)

        if intersecting_pairs:
            print(f"Found {len(intersecting_pairs)} intersecting triangle pairs.")
            # Register highlights in Polyscope
            register_intersection_highlights(mesh1, mesh2, intersecting_pairs)
        else:
            print("No intersections found.")
            # Remove any existing highlights
            if ps.has_surface_mesh("Intersecting Triangles Mesh 1"):
                ps.remove_surface_mesh("Intersecting Triangles Mesh 1")
            if ps.has_surface_mesh("Intersecting Triangles Mesh 2"):
                ps.remove_surface_mesh("Intersecting Triangles Mesh 2")
        
        # Optionally, compute and register a bounding box around the intersection
        if intersecting_pairs:
            # Extract vertices from intersecting triangles
            intersect_vertices1 = mesh1.triangles[[pair[0] for pair in intersecting_pairs]].reshape(-1, 3)
            intersect_vertices2 = mesh2.triangles[[pair[1] for pair in intersecting_pairs]].reshape(-1, 3)
            all_intersect_vertices = np.vstack((intersect_vertices1, intersect_vertices2))
            
            # Compute bounding box
            min_corner = np.min(all_intersect_vertices, axis=0)
            max_corner = np.max(all_intersect_vertices, axis=0)
            print("Min Corner:", min_corner)
            print("Max Corner:", max_corner)
            register_bounding_box(min_corner, max_corner, "Intersection Box", INTERSECT_COLOR1)
        else:
            remove_bounding_box("Intersection Box")
    else:
        # Remove any existing highlights
        if ps.has_surface_mesh("Intersecting Triangles Mesh 1"):
            ps.remove_surface_mesh("Intersecting Triangles Mesh 1")
        if ps.has_surface_mesh("Intersecting Triangles Mesh 2"):
            ps.remove_surface_mesh("Intersecting Triangles Mesh 2")
        remove_bounding_box("Intersection Box")

In [ ]:
def user_callback():
    """User interface for controlling transformations and toggles."""
    global scale1, rotation1, translation1, scale2, rotation2, translation2, show_intersection_box

    def transformation_ui(label_prefix, scale, rotation, translation, update_func):
        """Reusable UI for controlling transformations of a mesh."""
        # Scale
        psim.Text(f"{label_prefix} Controls:")
        changed, new_scale = psim.SliderFloat3(f"{label_prefix} Scale", scale, 0.1, 10.0, "%.2f")
        if changed:
            scale[:] = new_scale
            update_func()

        # Rotation
        changed, new_rotation = psim.SliderFloat3(f"{label_prefix} Rotation (deg)", rotation, -180.0, 180.0, "%.2f")
        if changed:
            rotation[:] = new_rotation
            update_func()

        # Translation
        changed, new_translation = psim.SliderFloat3(f"{label_prefix} Translation", translation, -10.0, 10.0, "%.2f")
        if changed:
            translation[:] = new_translation
            update_func()

    # Transformation controls for Mesh 1
    transformation_ui("Mesh 1", scale1, rotation1, translation1, update_geometry)

    # Transformation controls for Mesh 2
    transformation_ui("Mesh 2", scale2, rotation2, translation2, update_geometry)

    # Intersection toggle
    changed, new_value = psim.Checkbox("Show Intersection Highlights", show_intersection_box)
    if changed:
        show_intersection_box = new_value
        update_geometry()

In [ ]:
def main():
    global points1, points2, tetra_cells1, tetra_cells2, surface_faces1, surface_faces2

    # Verify mesh files exist
    verify_mesh_files_exist(MESH_FILE1, MESH_FILE2)

    # Load meshes
    points1, tetra_cells1 = load_mesh(MESH_FILE1)
    points2, tetra_cells2 = load_mesh(MESH_FILE2)

    # Extract boundary faces
    surface_faces1 = extract_boundary_faces(tetra_cells1)
    surface_faces2 = extract_boundary_faces(tetra_cells2)

    # Extract internal tetrahedra
    internal_tetrahedra1 = extract_internal_tetrahedra(tetra_cells1, surface_faces1)
    internal_tetrahedra2 = extract_internal_tetrahedra(tetra_cells2, surface_faces2)

    # Initialize Polyscope
    ps.init()
    ps.set_navigation_style("turntable")
    ps.set_transparency_mode("pretty")

    # Register external surface meshes with distinct colors
    ps.register_surface_mesh("External Surface Mesh 1", points1, surface_faces1, color=SURFACE_COLOR1)  # Blue
    ps.register_surface_mesh("External Surface Mesh 2", points2, surface_faces2, color=SURFACE_COLOR2)  # Red

    # Register internal volume meshes with transparency
    register_volume_mesh("Internal Tetrahedra 1", points1, internal_tetrahedra1, color=[0.0, 1.0, 0.0, 0.5])  # Semi-transparent Green
    register_volume_mesh("Internal Tetrahedra 2", points2, internal_tetrahedra2, color=[1.0, 1.0, 0.0, 0.5])  # Semi-transparent Yellow

    # Set user callback and update geometry
    ps.set_user_callback(user_callback)
    update_geometry()

    # Show Polyscope
    ps.show()


In [ ]:
if __name__ == "__main__":
    main()